### Attempting to remove stopwords before splitting the line lyrics by Token
#### Unsuccessful

In [1]:
# Import libraries and upload OHCO (by line) #
import pandas as pd
import numpy as np
import os

df = pd.read_csv('artist_song_verse_line_OHCO_df.csv') # includes corpus with no duplicate song names

In [2]:
df.head()

,artist_id,song_id,verse_num,line_num,Line
0,0,0,0,0,"Look at her face, it's a wonderful face"
1,0,0,0,1,And it means something special to me
2,0,0,0,2,Look at the way that she smiles when she sees me
3,0,0,0,3,How lucky can one fellow be?
4,0,0,1,0,"She's just my kind of girl, she makes me feel ..."


In [3]:
# Try the method on a subset first
sub = df[df['artist_id']==0]

In [4]:
sub=sub.set_index(['artist_id', 'song_id','verse_num','line_num'])

In [19]:
TOKEN_PAT = r'(\W+)'
tokens = sub.Line.str.split(TOKEN_PAT, expand=True)\
    .stack()\
    .to_frame()\
    .rename(columns={0:'token_str'})
tokens.index.names = ['artist_id', 'song_id','verse_num','line_num','token_num']
tokens['punc'] = tokens.token_str.str.match(r'^\W*$').astype('int')

In [20]:
tokens.head()

token_str  punc
artist_id song_id verse_num line_num token_num                
0         0       0         0        0              Look     0
                                     1                       1
                                     2                at     0
                                     3                       1
                                     4               her     0

In [21]:
#remove punctuation
no_punc = tokens[tokens.punc == 0]

In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
stemmer = nltk.stem.porter.PorterStemmer()
no_punc['port_stem'] = no_punc.token_str.apply(lambda x: stemmer.stem(x))

C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [28]:
no_stop=pd.DataFrame(no_punc[-no_punc.port_stem.isin(stop)])

In [55]:
no_stop.head()

token_str  punc port_stem
artist_id song_id verse_num line_num token_num                           
0         0       0         0        0               Look     0      look
                                     6               face     0      face
                                     14         wonderful     0    wonder
                                     16              face     0      face
                            1        4              means     0      mean

In [42]:
token_counts = no_stop[tokens.punc == 0].port_stem.str.lower().value_counts()

C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [43]:
token_counts.head()

i       1106
love     166
la       156
like     126
know     116
Name: port_stem, dtype: int64

In [44]:
vocab = no_stop[tokens.punc == 0].port_stem.value_counts().to_frame()\
    .reset_index()\
    .rename(columns={'index':'port_stem', 'port_stem':'n'})
vocab = vocab.sort_values('port_stem').reset_index()
vocab.index.name = 'term_id'
vocab = vocab.drop('index', 1)

C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [47]:
vocab.sort_values('n', ascending=False).head(25)

,port_stem,n
term_id,,
12,I,1106
849,love,166
782,la,139
813,like,126
779,know,116
515,feel,115
978,one,113
616,go,106
1525,wa,97


In [49]:
K=no_stop

In [62]:
K

token_str  punc port_stem  \
artist_id song_id verse_num line_num token_num                              
0         0       0         0        0               Look     0      look   
                                     6               face     0      face   
                                     14         wonderful     0    wonder   
                                     16              face     0      face   
                            1        4              means     0      mean   
                                     6          something     0    someth   
                                     8            special     0   special   
                            2        0               Look     0      look   
                                     6                way     0       way   
                                     12            smiles     0     smile   
                                     18              sees     0       see   
                            3        2              lucky     0     lucki   
                                     6                one     0       one   
                                     8             fellow     0    fellow   
                  1         0        8               kind     0      kind   
                                     12              girl     0      girl   
                                     16             makes     0      make   
                                     20              feel     0      feel   
                                     22              fine     0      fine   
                            1        2              could     0     could   
                                     4               ever     0      ever   
                                     6            believe     0    believ   
                                     12             could     0     could   
                                     16              mine     0      mine   
                            2        8               kind     0      kind   
                                     12              girl     0      girl   
                                     14           without     0   without   
                                     18                 I     0         I   
                                     22              blue     0      blue   
                            3        6               ever     0      ever   
...                                                   ...   ...       ...   
          86      11        3        10              done     0      done   
                  12        0        0                  I     0         I   
                                     4            missing     0      miss   
                                     10               fun     0       fun   
                            1        0               Baby     0      babi   
                                     4                owe     0       owe   
                                     8                one     0       one   
                            2        0                Aha     0       aha   
                                     2                 mm     0        mm   
                            3        0                 It     0        It   
                                     6               turn     0      turn   
                                     10               run     0       run   
                  13        0        0                  A     0         A   
                                     2             chance     0     chanc   
                                     6               feel     0      feel   
                                     10               sun     0       sun   
                            1        0               Baby     0      babi   
                                     4                owe     0       owe   
                                     8                one     0       one   
                            2        0      

In [56]:
K['term_id'] = K['port_stem'].map(V.reset_index().set_index('port_stem').term_id).fillna(-1).astype('int')

In [50]:
V=vocab

In [63]:
OHCO = ['artist_id', 'song_id', 'verse_num', 'line_num','token_num']

In [66]:
SONG=OHCO[1:2]

In [67]:
SONG

['song_id']

In [68]:
BOW = K.groupby(SONG+['term_id'])['term_id'].count()

In [70]:
BOW.head()

song_id  term_id
0        12         6
         40         1
         157        2
         182        2
         322        8
Name: term_id, dtype: int64

In [71]:
DTM = BOW.unstack().fillna(0)

In [72]:
DTM

term_id,0,1,2,3,4,5,6,7,8,9,...,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610
song_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
alpha = .000001 # We introduce an arbitrary smoothing value
alpha_sum = alpha * V.shape[0]
TF = DTM.apply(lambda x: (x + alpha) / (x.sum() + alpha_sum), axis=1)

In [74]:
TF.head()

term_id,0,1,2,3,4,5,6,7,8,9,...,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610
song_id,,,,,,,,,,,,,,,,,,,,,
0,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,...,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08,1.449242e-08
1,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,...,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09,6.849239e-09
2,6.369361e-09,6.369361e-09,6.369361e-09,3.821617e-02,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,...,6.369361e-09,6.369361e-09,1.910809e-02,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09,6.369361e-09
3,1.351337e-02,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09,1.351337e-02,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09,...,6.756683e-09,6.756683e-09,6.756690e-03,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09,6.756683e-09
4,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,...,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08,1.724090e-08


In [76]:
N_docs = DTM.shape[0]
V['df'] = DTM[DTM > 0].count()
TFIDF = TF * np.log2(N_docs / V['df'])

In [79]:
TFIDF.head()

term_id,0,1,2,3,4,5,6,7,8,9,...,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610
song_id,,,,,,,,,,,,,,,,,,,,,
0,4.523105e-08,7.888140e-08,7.888140e-08,4.989657e-08,7.040388e-08,7.888140e-08,9.337381e-08,7.040388e-08,9.337381e-08,9.337381e-08,...,9.337381e-08,9.337381e-08,2.781647e-08,5.972347e-08,7.888140e-08,9.337381e-08,5.268846e-08,9.337381e-08,7.040388e-08,9.337381e-08
1,2.137658e-08,3.728002e-08,3.728002e-08,2.358154e-08,3.327348e-08,3.728002e-08,4.412926e-08,3.327348e-08,4.412926e-08,4.412926e-08,...,4.412926e-08,4.412926e-08,1.314630e-08,2.822582e-08,3.728002e-08,4.412926e-08,2.490102e-08,4.412926e-08,3.327348e-08,4.412926e-08
2,1.987888e-08,3.466807e-08,3.466807e-08,1.315761e-01,3.094224e-08,3.466807e-08,4.103744e-08,3.094224e-08,4.103744e-08,4.103744e-08,...,4.103744e-08,4.103744e-08,3.667572e-02,2.624824e-08,3.466807e-08,4.103744e-08,2.315638e-08,4.103744e-08,3.094224e-08,4.103744e-08
3,4.217545e-02,3.677624e-08,3.677624e-08,2.326288e-08,3.282384e-08,7.355253e-02,4.353293e-08,3.282384e-08,4.353293e-08,4.353293e-08,...,4.353293e-08,4.353293e-08,1.296867e-02,2.784440e-08,3.677624e-08,4.353293e-08,2.456452e-08,4.353293e-08,3.282384e-08,4.353293e-08
4,5.380912e-08,9.384125e-08,9.384125e-08,5.935945e-08,8.375597e-08,9.384125e-08,1.110821e-07,8.375597e-08,1.110821e-07,1.110821e-07,...,1.110821e-07,1.110821e-07,3.309187e-08,7.105002e-08,9.384125e-08,1.110821e-07,6.268082e-08,1.110821e-07,8.375597e-08,1.110821e-07


In [81]:
THM = -(TF * np.log2(TF))
TFTH = TF.apply(lambda x: x * THM.sum(), 1)

In [82]:
V['tf_sum'] = TF.sum()
V['tf_mean'] = TF.mean()
V['tf_max'] = TF.max()
V['tfidf_sum'] = TFIDF.sum()
V['tfidf_mean'] = TFIDF.mean()
V['tfidf_max'] = TFIDF.max()
V['tfth_sum'] = TFTH.sum()
V['tfth_mean'] = TFTH.mean()
V['tfth_max'] = TFTH.max()
V['th_sum'] = THM.sum()
V['th_mean'] = THM.mean()
V['th_max'] = THM.max()

In [83]:
V

,port_stem,n,df,tf_sum,tf_mean,tf_max,tfidf_sum,tfidf_mean,tfidf_max,tfth_sum,tfth_mean,tfth_max,th_sum,th_mean,th_max
term_id,,,,,,,,,,,,,,,
0,A,22,10,0.136796,0.001572,0.027624,0.426943,0.004907,0.086215,0.112644,0.001295,0.022747,0.823447,0.009465,0.143036
1,Ah,3,2,0.017084,0.000196,0.010417,0.092987,0.001069,0.056697,0.001995,0.000023,0.001217,0.116803,0.001343,0.068593
2,Am,2,2,0.033155,0.000381,0.023809,0.180461,0.002074,0.129589,0.006346,0.000073,0.004557,0.191407,0.002200,0.128385
3,As,16,8,0.116348,0.001337,0.038216,0.400579,0.004604,0.131576,0.078870,0.000907,0.025906,0.677882,0.007792,0.179986
4,At,3,3,0.019105,0.000220,0.008696,0.092813,0.001067,0.042243,0.002645,0.000030,0.001204,0.138435,0.001591,0.059525
5,By,3,2,0.018862,0.000217,0.013513,0.102663,0.001180,0.073553,0.002344,0.000027,0.001680,0.124287,0.001429,0.083911
6,De,3,1,0.012932,0.000149,0.012931,0.083318,0.000958,0.083313,0.001049,0.000012,0.001049,0.081135,0.000933,0.081116
7,Do,4,3,0.025130,0.000289,0.013245,0.122083,0.001403,0.064343,0.004281,0.000049,0.002257,0.170369,0.001958,0.082627
8,Dr,1,1,0.005918,0.000068,0.005917,0.038128,0.000438,0.038124,0.000259,0.000003,0.000259,0.043811,0.000504,0.043792


In [85]:
#Find significant words

In [87]:
TOPS = pd.DataFrame(index=range(10))
for m in ['tf','tfidf','tfth', 'th']:
    for n in ['mean','max']:
        key = '{}_{}'.format(m,n)
        TOPS[key] = V.sort_values(key, ascending=False).port_stem.head(10).tolist()

In [88]:
TOPS

,tf_mean,tf_max,tfidf_mean,tfidf_max,tfth_mean,tfth_max,th_mean,th_max
0,I,I,la,la,I,I,I,la
1,love,la,love,hey,love,love,love,hey
2,know,hey,hey,gimm,know,know,like,I
3,like,honey,We,bale,like,la,know,honey
4,feel,gimm,gimm,waterloo,feel,make,feel,gimm
5,go,sweet,dumb,helen,go,one,go,sweet
6,one,bale,let,honey,one,Oh,see,bale
7,wa,pick,gonna,jeani,wa,We,wa,pick
8,la,waterloo,danc,pick,see,let,one,waterloo
9,see,ring,Oh,andant,make,thi,day,ring


In [90]:
#Compare Chapters
#Create Songs Table

In [93]:
def get_songs(tokens, div_names, doc_str = 'term_id', sep='', flatten=False, 
             index_only=False):
    
    if not index_only:
        docs = tokens.groupby(div_names)[doc_str]\
          .apply(lambda x: x.str.cat(sep=sep))
        docs.columns = ['doc_content']
    else:
        docs = tokens.groupby(div_names)[doc_str].apply(lambda x: x.tolist())

    if flatten:
        docs = docs.reset_index().drop(div_names, 1)
    
    return docs

In [94]:
D = get_songs(K, SONG, 'port_stem', sep=" ")\
    .to_frame().rename(columns={'port_stem':'song_str'})

In [95]:
D

,song_str
song_id,
0,look face wonder face mean someth special look...
1,take easi pleas touch gentli like summer even ...
2,I never know whi I go whi I put lousi rotten s...
3,make somebodi happi question give take learn s...
4,well hoot holler make mad I alway heel holi ch...
5,street sing shout stay aliv though citi dead h...
6,chiquitita tell wrong enchain sorrow In eye ho...
7,chang move circl I see face dream smile laugh ...
8,I hear work practis hard play night day wo oh ...
